In [1]:
#!pip install python-dotenv decorator==4.3 networkx
#!pip install pycoingecko
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pprint


%matplotlib inline

In [2]:
load_dotenv('../.env')

True

In [3]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [4]:
coin_market = cg.get_coins_markets(vs_currency='usd')
coin_market_df= pd.DataFrame(coin_market, columns=['id', 'current_price', 'high_24h' , 'low_24h'])
coin_market_df.set_index('id', inplace=True)
coin_market_df

,current_price,high_24h,low_24h
id,,,
bitcoin,56853.000000,57860.000000,55820.000000
ethereum,3487.600000,3615.350000,3480.920000
tether,1.000000,1.010000,0.996354
cardano,2.120000,2.250000,2.110000
binancecoin,406.840000,423.360000,405.570000
...,...,...,...
iostoken,0.052151,0.055634,0.052187
spell-token,0.019666,0.020798,0.017167
zilliqa,0.090922,0.099046,0.091120


In [5]:
btc_history = cg.get_coin_market_chart_by_id(id= 'bitcoin', vs_currency = 'usd', days = 'max')
btc_history_df = pd.DataFrame(btc_history)

In [6]:
def extract_volume(x):
    return x[0]
def extract_price(x):
    return x[1]

btc_history_df["volume"] = btc_history_df['prices'].apply(extract_volume)
btc_history_df["price"] = btc_history_df['prices'].apply(extract_price)
btc_history_df["market_cap"] = btc_history_df['market_caps'].apply(extract_price)
btc_history_df["total_volume"] = btc_history_df['total_volumes'].apply(extract_price)

In [54]:
btc_history_df = btc_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
btc_history_df["date"] = pd.to_datetime(btc_history_df["volume"], unit="ms")
btc_history_df.set_index('date', inplace=True)
btc_history_df.drop(columns='volume', inplace=True)
btc_history_df.head()

,price,market_cap,total_volume
date,,,
2013-04-28,135.30,1.500518e+09,0.0
2013-04-29,141.96,1.575032e+09,0.0
2013-04-30,135.30,1.501657e+09,0.0
2013-05-01,117.00,1.298952e+09,0.0
2013-05-02,103.43,1.148668e+09,0.0


In [8]:
eth_history = cg.get_coin_market_chart_by_id(id= 'ethereum', vs_currency = 'usd', days = 'max')
eth_history_df = pd.DataFrame(eth_history)


In [9]:
eth_history_df["volume"] = eth_history_df['prices'].apply(extract_volume)
eth_history_df["price"] = eth_history_df['prices'].apply(extract_price)
eth_history_df["market_cap"] = eth_history_df['market_caps'].apply(extract_price)
eth_history_df["total_volume"] = eth_history_df['total_volumes'].apply(extract_price)

In [10]:
eth_history_df = eth_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
eth_history_df["date"] = pd.to_datetime(eth_history_df["volume"], unit="ms")
eth_history_df.set_index('date', inplace=True)
eth_history_df.drop(columns='volume', inplace=True)
eth_history_df.head()

,volume,price,market_cap,total_volume
date,,,,
2015-08-07,1438905600000,2.831620,0.000000e+00,9.062200e+04
2015-08-08,1438992000000,1.330750,8.033948e+07,3.680700e+05
2015-08-10,1439164800000,0.687586,4.155631e+07,4.004641e+05
2015-08-11,1439251200000,1.067379,6.453901e+07,1.518998e+06
2015-08-12,1439337600000,1.256613,7.601326e+07,2.073893e+06


In [11]:
lte_history = cg.get_coin_market_chart_by_id(id= 'litecoin', vs_currency = 'usd', days = 'max')
lte_history_df = pd.DataFrame(lte_history)

In [12]:
lte_history_df["volume"] = lte_history_df['prices'].apply(extract_volume)
lte_history_df["price"] = lte_history_df['prices'].apply(extract_price)
lte_history_df["market_cap"] = lte_history_df['market_caps'].apply(extract_price)
lte_history_df["total_volume"] = lte_history_df['total_volumes'].apply(extract_price)

In [13]:
lte_history_df = lte_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
lte_history_df["date"] = pd.to_datetime(lte_history_df["volume"], unit="ms")
lte_history_df.set_index('date', inplace=True)
lte_history_df.drop(columns='volume', inplace=True)
lte_history_df.head()

,volume,price,market_cap,total_volume
date,,,,
2013-04-28,1367107200000,4.29983,73773387.0,0.0
2013-04-29,1367193600000,4.35940,74936909.0,0.0
2013-04-30,1367280000000,4.18295,72037636.0,0.0
2013-05-01,1367366400000,3.64914,62957992.0,0.0
2013-05-02,1367452800000,3.38879,58565340.0,0.0


In [14]:
link_history = cg.get_coin_market_chart_by_id(id= 'chainlink', vs_currency = 'usd', days = 'max')
link_history_df = pd.DataFrame(link_history)

In [15]:
link_history_df["volume"] = link_history_df['prices'].apply(extract_volume)
link_history_df["price"] = link_history_df['prices'].apply(extract_price)
link_history_df["market_cap"] = link_history_df['market_caps'].apply(extract_price)
link_history_df["total_volume"] = link_history_df['total_volumes'].apply(extract_price)

In [16]:
link_history_df = link_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
link_history_df["date"] = pd.to_datetime(link_history_df["volume"], unit="ms")
link_history_df.set_index('date', inplace=True)
link_history_df.drop(columns='volume', inplace=True)
link_history_df.head()

,volume,price,market_cap,total_volume
date,,,,
2017-11-09,1510185600000,0.225377,7.888183e+07,3.190026e+06
2017-11-10,1510272000000,0.189036,6.616275e+07,2.121175e+06
2017-11-11,1510358400000,0.181223,6.342793e+07,2.229298e+06
2017-11-12,1510444800000,0.170013,5.950462e+07,5.361146e+06
2017-11-13,1510531200000,0.180691,6.324186e+07,2.942234e+06


In [17]:
ada_history = cg.get_coin_market_chart_by_id(id= 'cardano', vs_currency = 'usd', days = 'max')
ada_history_df = pd.DataFrame(ada_history)

,prices,market_caps,total_volumes
0,"[1508284800000, 0.02684535467621909]","[1508284800000, 696021404.3079604]","[1508284800000, 2351678.122306208]"
1,"[1508371200000, 0.026829950465728686]","[1508371200000, 695622018.2559936]","[1508371200000, 2815155.9695999967]"
2,"[1508457600000, 0.030299606216252807]","[1508457600000, 785580027.6424098]","[1508457600000, 8883473.082062634]"
3,"[1508544000000, 0.028587962639027336]","[1508544000000, 741202123.8797704]","[1508544000000, 5308856.917095766]"
4,"[1508630400000, 0.027796038407662104]","[1508630400000, 720669848.4724126]","[1508630400000, 2901876.1220860113]"
...,...,...,...
1450,"[1633564800000, 2.2133070922060374]","[1633564800000, 70967214662.64716]","[1633564800000, 2144512119.753268]"
1451,"[1633651200000, 2.279964070864212]","[1633651200000, 72987443161.14131]","[1633651200000, 2669936578.5982766]"
1452,"[1633737600000, 2.2306779663351755]","[1633737600000, 71588795681.69351]","[1633737600000, 1521104723.9480116]"
1453,"[1633824000000, 2.2694736192158076]","[1633824000000, 72764350708.77914]","[1633824000000, 1322555134.0419257]"


In [18]:
ada_history_df["volume"] = ada_history_df['prices'].apply(extract_volume)
ada_history_df["price"] = ada_history_df['prices'].apply(extract_price)
ada_history_df["market_cap"] = ada_history_df['market_caps'].apply(extract_price)
ada_history_df["total_volume"] = ada_history_df['total_volumes'].apply(extract_price)

In [55]:
ada_history_df = ada_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
ada_history_df["date"] = pd.to_datetime(ada_history_df["volume"], unit="ms")
ada_history_df.set_index('date', inplace=True)
ada_history_df.drop(columns='volume', inplace=True)
ada_history_df.head()

,price,market_cap,total_volume
date,,,
2017-10-18,0.026845,6.960214e+08,2.351678e+06
2017-10-19,0.026830,6.956220e+08,2.815156e+06
2017-10-20,0.030300,7.855800e+08,8.883473e+06
2017-10-21,0.028588,7.412021e+08,5.308857e+06
2017-10-22,0.027796,7.206698e+08,2.901876e+06


In [56]:
to_merge_dict = {
    'BTC': btc_history_df , 'ETH': eth_history_df,
    'LTE':lte_history_df,'LINK':link_history_df, 'ADA':ada_history_df }

In [57]:
#Coin Merged DataFrame
merged_coin_df = pd.concat(to_merge_dict.values(), keys=to_merge_dict.keys(), axis=1, join='inner')
merged_coin_df.head()

BTC                                        ETH  \
                  price    market_cap  total_volume         volume   
date                                                                 
2017-11-09  7186.292775  1.198168e+11  2.330694e+09  1510185600000   
2017-11-10  6656.839694  1.109994e+11  3.886426e+09  1510272000000   
2017-11-11  6365.397852  1.061461e+11  3.148370e+09  1510358400000   
2017-11-12  5866.513129  9.783217e+10  6.001018e+09  1510444800000   
2017-11-13  6423.091402  1.071254e+11  4.636292e+09  1510531200000   

                                                              LTE             \
                 price    market_cap  total_volume         volume      price   
date                                                                           
2017-11-09  322.329520  3.082374e+10  6.918154e+08  1510185600000  64.537577   
2017-11-10  301.354933  2.882416e+10  6.505058e+08  1510272000000  59.371318   
2017-11-11  316.258342  3.025603e+10  6.186761e+08  1510358400000  62.612170   
2017-11-12  314.166128  3.006218e+10  1.172944e+09  1510444800000  58.916112   
2017-11-13  315.343956  3.018143e+10  7.833661e+08  1510531200000  61.163042   

                                                 LINK                          \
              market_cap  total_volume         volume     price    market_cap   
date                                                                            
2017-11-09  3.469079e+09  2.345449e+08  1510185600000  0.225377  7.888183e+07   
2017-11-10  3.192341e+09  2.366416e+08  1510272000000  0.189036  6.616275e+07   
2017-11-11  3.367560e+09  2.256785e+08  1510358400000  0.181223  6.342793e+07   
2017-11-12  3.169837e+09  2.526936e+08  1510444800000  0.170013  5.950462e+07   
2017-11-13  3.291569e+09  1.543181e+08  1510531200000  0.180691  6.324186e+07   

                               ADA                              
            total_volume     price    market_cap  total_volume  
date                                                            
2017-11-09  3.190026e+06  0.032769  8.496170e+08  2.062814e+07  
2017-11-10  2.121175e+06  0.027293  7.076286e+08  6.563684e+06  
2017-11-11  2.229298e+06  0.027499  7.129560e+08  5.632138e+06  
2017-11-12  5.361146e+06  0.024522  6.357746e+08  7.550745e+06  
2017-11-13  2.942234e+06  0.025114  6.511399e+08  4.284486e+06

In [58]:
merged_coin_df.to_json('crypto_data.json')

In [26]:
#Alpaca Keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [27]:
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [28]:
aapl_data = api.get_barset('AAPL',timeframe='1D', limit=1000,).df
aapl_data.tail()

AAPL                                    
                             open      high     low   close    volume
time                                                                 
2021-10-05 00:00:00-04:00  139.49  142.2400  139.36  141.10  68998178
2021-10-06 00:00:00-04:00  139.58  142.1500  138.37  141.99  73830085
2021-10-07 00:00:00-04:00  143.00  144.2150  142.72  143.28  54652459
2021-10-08 00:00:00-04:00  144.04  144.1781  142.56  143.01  48459122
2021-10-11 00:00:00-04:00  142.30  144.8100  141.81  142.80  55706039

In [29]:
amzn_data = api.get_barset('AMZN',timeframe='1D', limit=1000,).df
amzn_data.head()

AMZN                                  
                             open     high     low   close   volume
time                                                               
2017-10-20 00:00:00-04:00  993.53  994.620  982.00  982.91  1406057
2017-10-23 00:00:00-04:00  986.73  986.775  962.50  966.34  2341953
2017-10-24 00:00:00-04:00  969.00  979.850  965.00  976.00  1744834
2017-10-25 00:00:00-04:00  978.00  984.440  966.24  972.91  2001000
2017-10-26 00:00:00-04:00  980.33  982.900  968.55  972.54  2373669

In [30]:
ba_data = api.get_barset('BA',timeframe='1D', limit=1000,).df
ba_data.head()

BA                                     
                             open      high     low     close   volume
time                                                                  
2017-10-20 00:00:00-04:00  260.36  264.8272  259.44  264.6900  2860789
2017-10-23 00:00:00-04:00  265.13  265.9800  262.14  262.3500  2385662
2017-10-24 00:00:00-04:00  263.42  267.2100  263.24  266.0500  3316040
2017-10-25 00:00:00-04:00  264.78  266.0000  254.50  258.4199  6031790
2017-10-26 00:00:00-04:00  259.72  261.4400  257.31  259.2700  2631747

In [31]:
nvda_data = api.get_barset('NVDA',timeframe='1D', limit=1000,).df
nvda_data.head()

NVDA                                  
                             open    high     low   close    volume
time                                                               
2017-10-20 00:00:00-04:00  198.49  199.59  196.54  196.86   9450691
2017-10-23 00:00:00-04:00  197.70  198.85  195.80  196.62   9054036
2017-10-24 00:00:00-04:00  196.99  198.74  195.94  198.67   8679881
2017-10-25 00:00:00-04:00  197.08  199.22  191.17  193.64  17900060
2017-10-26 00:00:00-04:00  194.91  196.16  193.81  195.59   6727585

In [32]:
brk_data = api.get_barset('BRK.A',timeframe='1D', limit=1000,).df
brk_data.head()

BRK.A                                      
                               open      high        low     close volume
time                                                                     
2017-09-27 00:00:00-04:00  274260.0  275745.0  273800.62  274940.0    159
2017-09-28 00:00:00-04:00  275870.0  275945.0  273825.00  274770.0    101
2017-09-29 00:00:00-04:00  274900.0  274900.0  273755.00  274900.0    198
2017-10-02 00:00:00-04:00  274999.0  278010.0  274500.00  278010.0    225
2017-10-03 00:00:00-04:00  278200.0  279660.0  277000.00  278630.0    214

In [33]:
gs_data = api.get_barset('GS',timeframe='1D', limit=1000,).df
gs_data.head()

GS                                  
                             open    high      low   close   volume
time                                                               
2017-10-20 00:00:00-04:00  242.95  245.25  240.670  244.74  2310872
2017-10-23 00:00:00-04:00  244.39  245.33  241.381  242.16  1475595
2017-10-24 00:00:00-04:00  243.60  246.25  243.000  244.87  1503543
2017-10-25 00:00:00-04:00  246.16  246.50  240.800  241.71  2072140
2017-10-26 00:00:00-04:00  242.33  244.00  241.530  241.72  1430800

In [34]:
cop_data = api.get_barset('COP',timeframe='1D', limit=1000,).df
cop_data.head()

COP                                 
                            open   high      low   close   volume
time                                                             
2017-10-20 00:00:00-04:00  49.90  50.40  49.5601  50.250  4856606
2017-10-23 00:00:00-04:00  50.28  50.75  49.9300  50.270  3860560
2017-10-24 00:00:00-04:00  50.48  51.32  50.4200  50.995  6532059
2017-10-25 00:00:00-04:00  51.06  51.13  49.7500  49.960  5799954
2017-10-26 00:00:00-04:00  50.39  51.77  49.7900  51.450  6276649

In [35]:
gild_data = api.get_barset('GILD',timeframe='1D', limit=1000,).df
gild_data.head()

GILD                              
                            open   high    low  close   volume
time                                                          
2017-10-20 00:00:00-04:00  81.83  81.95  80.45  81.21  7203471
2017-10-23 00:00:00-04:00  81.25  81.32  80.09  80.17  5181910
2017-10-24 00:00:00-04:00  80.18  80.38  78.80  80.09  5695051
2017-10-25 00:00:00-04:00  80.08  80.08  78.69  79.90  6835318
2017-10-26 00:00:00-04:00  78.44  79.50  77.00  77.84  9619233

In [36]:
ccoi_data = api.get_barset('CCOI',timeframe='1D', limit=1000,).df
ccoi_data.head()

CCOI                              
                            open   high     low  close  volume
time                                                          
2017-10-18 00:00:00-04:00  50.05  50.20  49.600  49.70  202954
2017-10-19 00:00:00-04:00  49.65  50.05  47.800  48.41  128608
2017-10-20 00:00:00-04:00  48.89  50.50  47.880  49.65  204367
2017-10-23 00:00:00-04:00  49.85  50.20  49.025  49.30  142925
2017-10-24 00:00:00-04:00  49.65  50.10  49.200  49.90  158656

In [37]:
proc_data = api.get_barset('PG',timeframe='1D', limit=1000,).df
proc_data.head()

PG                               
                            open   high    low  close    volume
time                                                           
2017-10-20 00:00:00-04:00  89.72  89.96  87.59  88.22  16520870
2017-10-23 00:00:00-04:00  88.63  88.86  86.87  87.29  10177891
2017-10-24 00:00:00-04:00  87.37  87.42  86.78  87.00   7370493
2017-10-25 00:00:00-04:00  86.90  87.11  86.28  86.83   6587502
2017-10-26 00:00:00-04:00  87.19  87.77  87.10  87.49   4597174

In [38]:
# aapl_data.dropna(inplace=True)
# amzn_data.dropna(inplace=True)
# ba_data.dropna(inplace=True)
# nvda_data.dropna(inplace=True)
# brk_data.dropna(inplace=True)
# gs_data.dropna(inplace=True)
# cop_data.dropna(inplace=True) 
# gild_data.dropna(inplace=True)
# ccoi_data.dropna(inplace=True)
# proc_data.dropna(inplace=True)

In [39]:
merged_stock_df= pd.concat([
    aapl_data, amzn_data, ba_data, nvda_data, brk_data,
    gs_data, cop_data, gild_data, ccoi_data, proc_data], axis='columns', join='inner')
#merged_stock_df = pd.concat(to_merge_dict.values())
#merged_stock_df.dropna(inplace=True)
merged_stock_df.head()

AAPL                                        AMZN  \
                             open      high     low   close    volume    open   
time                                                                            
2017-10-20 00:00:00-04:00  156.61  157.7500  155.96  156.25  18427186  993.53   
2017-10-23 00:00:00-04:00  156.89  157.6900  155.50  156.20  16418890  986.73   
2017-10-24 00:00:00-04:00  156.29  157.4200  156.20  157.09  14016780  969.00   
2017-10-25 00:00:00-04:00  156.91  157.5500  155.27  156.41  16016689  978.00   
2017-10-26 00:00:00-04:00  157.23  157.8295  156.78  157.42  12374254  980.33   

                                                             ...   CCOI        \
                              high     low   close   volume  ...   open  high   
time                                                         ...                
2017-10-20 00:00:00-04:00  994.620  982.00  982.91  1406057  ...  48.89  50.5   
2017-10-23 00:00:00-04:00  986.775  962.50  966.34  2341953  ...  49.85  50.2   
2017-10-24 00:00:00-04:00  979.850  965.00  976.00  1744834  ...  49.65  50.1   
2017-10-25 00:00:00-04:00  984.440  966.24  972.91  2001000  ...  50.15  51.0   
2017-10-26 00:00:00-04:00  982.900  968.55  972.54  2373669  ...  49.42  51.6   

                                                     PG                       \
                              low  close  volume   open   high    low  close   
time                                                                           
2017-10-20 00:00:00-04:00  47.880  49.65  204367  89.72  89.96  87.59  88.22   
2017-10-23 00:00:00-04:00  49.025  49.30  142925  88.63  88.86  86.87  87.29   
2017-10-24 00:00:00-04:00  49.200  49.90  158656  87.37  87.42  86.78  87.00   
2017-10-25 00:00:00-04:00  49.180  49.33  190010  86.90  87.11  86.28  86.83   
2017-10-26 00:00:00-04:00  49.130  51.45  138453  87.19  87.77  87.10  87.49   

                                     
                             volume  
time                                 
2017-10-20 00:00:00-04:00  16520870  
2017-10-23 00:00:00-04:00  10177891  
2017-10-24 00:00:00-04:00   7370493  
2017-10-25 00:00:00-04:00   6587502  
2017-10-26 00:00:00-04:00   4597174  

[5 rows x 50 columns]

In [40]:
merged_stock_df.to_json('stock_data.json')

In [41]:
#Bond Data
#iSHares 20 year plus treasury
bond_data = api.get_barset(['TLT', 'VOOG', 'VGLT', 'VMBS', 'IWF', 'IGSB', 'GOVT'],timeframe='1D', limit=1000,).df


In [42]:
bond_data.dropna(inplace=True)
bond_data.head()

GOVT                                    IGSB  \
                            open    high    low  close    volume    open   
time                                                                       
2017-10-19 00:00:00-04:00  25.25  25.265  25.22  25.23  689206.0  52.635   
2017-10-20 00:00:00-04:00  25.13  25.170  25.13  25.15  651942.0  52.605   
2017-10-23 00:00:00-04:00  25.15  25.180  25.15  25.17  472528.0  52.625   
2017-10-24 00:00:00-04:00  25.11  25.135  25.10  25.12  636516.0  52.580   
2017-10-25 00:00:00-04:00  25.07  25.100  25.05  25.09  575142.0  52.600   

                                                              ...   VMBS  \
                             high     low   close     volume  ...   open   
time                                                          ...          
2017-10-19 00:00:00-04:00  52.645  52.605  52.630   529644.0  ...  52.77   
2017-10-20 00:00:00-04:00  52.614  52.595  52.605   545736.0  ...  52.65   
2017-10-23 00:00:00-04:00  52.640  52.601  52.635   856312.0  ...  52.68   
2017-10-24 00:00:00-04:00  52.625  52.580  52.610  1359706.0  ...  52.67   
2017-10-25 00:00:00-04:00  52.605  52.570  52.595   413946.0  ...  52.56   

                                                                VOOG  \
                              high      low  close    volume    open   
time                                                                   
2017-10-19 00:00:00-04:00  52.8200  52.7300  52.73  424189.0  131.10   
2017-10-20 00:00:00-04:00  52.6892  52.6301  52.64  286452.0  132.00   
2017-10-23 00:00:00-04:00  52.7200  52.6700  52.72  262840.0  132.30   
2017-10-24 00:00:00-04:00  52.6900  52.6100  52.62  326018.0  131.59   
2017-10-25 00:00:00-04:00  52.8600  52.5300  52.57  404450.0  131.26   

                                                                
                               high       low   close   volume  
time                                                            
2017-10-19 00:00:00-04:00  131.5000  130.8000  131.50  32793.0  
2017-10-20 00:00:00-04:00  132.0800  131.7100  131.95  32071.0  
2017-10-23 00:00:00-04:00  132.3000  131.2500  131.33  29681.0  
2017-10-24 00:00:00-04:00  131.5900  131.2099  131.41  39516.0  
2017-10-25 00:00:00-04:00  131.4175  130.2600  130.94  35633.0  

[5 rows x 35 columns]

In [43]:
bond_data.to_json('bond_data.json')

In [44]:
#Machine Learning for Investments
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [45]:
#Calcualtions for Daily Return which help with creating a target for ML
merged_stock_df.loc[:, ('AAPL', 'Daily_Returns')] = merged_stock_df['AAPL']['close'].pct_change()
merged_stock_df.loc[:,('AMZN','Daily_Returns')] = merged_stock_df['AMZN']['close'].pct_change()
merged_stock_df.loc[:,('BA','Daily_Returns')] = merged_stock_df['BA']['close'].pct_change()
merged_stock_df.loc[:,('NVDA','Daily_Returns')] = merged_stock_df['NVDA']['close'].pct_change()
merged_stock_df.loc[:,('BRK.A','Daily_Returns')] = merged_stock_df['BRK.A']['close'].pct_change()
merged_stock_df.loc[:,('GS','Daily_Returns')] = merged_stock_df['GS']['close'].pct_change()
merged_stock_df.loc[:,('COP','Daily_Returns')] = merged_stock_df['COP']['close'].pct_change()
merged_stock_df.loc[:,('GILD','Daily_Returns')] = merged_stock_df['GILD']['close'].pct_change()
merged_stock_df.loc[:,('CCOI','Daily_Returns')] = merged_stock_df['CCOI']['close'].pct_change()
merged_stock_df.loc[:,('PG','Daily_Returns')] =merged_stock_df['PG']['close'].pct_change()
merged_stock_return_df=  merged_stock_df.copy()
merged_stock_return_df.dropna(inplace=True)

In [46]:
merged_stock_return_df.loc[:, ('AAPL', 'Daily_Returns')] = merged_stock_df['AAPL']['close'].pct_change()
merged_stock_return_df.loc[:,('AMZN','Daily_Returns')] = merged_stock_df['AMZN']['close'].pct_change()
merged_stock_return_df.loc[:,('BA','Daily_Returns')] = merged_stock_df['BA']['close'].pct_change()
merged_stock_return_df.loc[:,('NVDA','Daily_Returns')] = merged_stock_df['NVDA']['close'].pct_change()
merged_stock_return_df.loc[:,('BRK.A','Daily_Returns')] = merged_stock_df['BRK.A']['close'].pct_change()
merged_stock_return_df.loc[:,('GS','Daily_Returns')] = merged_stock_df['GS']['close'].pct_change()
merged_stock_return_df.loc[:,('COP','Daily_Returns')] = merged_stock_df['COP']['close'].pct_change()
merged_stock_return_df.loc[:,('GILD','Daily_Returns')] = merged_stock_df['GILD']['close'].pct_change()
merged_stock_return_df.loc[:,('CCOI','Daily_Returns')] = merged_stock_df['CCOI']['close'].pct_change()
merged_stock_return_df.loc[:,('PG','Daily_Returns')] = merged_stock_df['PG']['close'].pct_change()

merged_stock_return_df.dropna(inplace=True)


In [47]:
# l = []
# for x in dict(merged_stock_df.columns):
#     l.append((x, 'close'))
# new_merged_stock_df = merged_stock_df[l]
#new_merged_stock_df.head()

In [48]:
merged_stock_return_df.head()

AAPL                                      \
                             open      high     low   close    volume   
time                                                                    
2017-10-23 00:00:00-04:00  156.89  157.6900  155.50  156.20  16418890   
2017-10-24 00:00:00-04:00  156.29  157.4200  156.20  157.09  14016780   
2017-10-25 00:00:00-04:00  156.91  157.5500  155.27  156.41  16016689   
2017-10-26 00:00:00-04:00  157.23  157.8295  156.78  157.42  12374254   
2017-10-27 00:00:00-04:00  159.29  163.6000  158.70  163.03  36735155   

                              AMZN                                        ...  \
                              open      high      low    close    volume  ...   
time                                                                      ...   
2017-10-23 00:00:00-04:00   986.73   986.775   962.50   966.34   2341953  ...   
2017-10-24 00:00:00-04:00   969.00   979.850   965.00   976.00   1744834  ...   
2017-10-25 00:00:00-04:00   978.00   984.440   966.24   972.91   2001000  ...   
2017-10-26 00:00:00-04:00   980.33   982.900   968.55   972.54   2373669  ...   
2017-10-27 00:00:00-04:00  1057.03  1105.580  1050.55  1100.99  11137045  ...   

                                   AAPL          AMZN            BA  \
                          Daily_Returns Daily_Returns Daily_Returns   
time                                                                  
2017-10-23 00:00:00-04:00     -0.000320     -0.016858     -0.008841   
2017-10-24 00:00:00-04:00      0.005698      0.009996      0.014103   
2017-10-25 00:00:00-04:00     -0.004329     -0.003166     -0.028679   
2017-10-26 00:00:00-04:00      0.006457     -0.000380      0.003290   
2017-10-27 00:00:00-04:00      0.035637      0.132077     -0.011417   

                                   NVDA         BRK.A            GS  \
                          Daily_Returns Daily_Returns Daily_Returns   
time                                                                  
2017-10-23 00:00:00-04:00     -0.001219      0.002984     -0.010542   
2017-10-24 00:00:00-04:00      0.010426      0.001027      0.011191   
2017-10-25 00:00:00-04:00     -0.025318     -0.004534     -0.012905   
2017-10-26 00:00:00-04:00      0.010070     -0.000159      0.000041   
2017-10-27 00:00:00-04:00      0.031494     -0.001271      0.000207   

                                    COP          GILD          CCOI  \
                          Daily_Returns Daily_Returns Daily_Returns   
time                                                                  
2017-10-23 00:00:00-04:00      0.000398     -0.012806     -0.007049   
2017-10-24 00:00:00-04:00      0.014422     -0.000998      0.012170   
2017-10-25 00:00:00-04:00     -0.020296     -0.002372     -0.011423   
2017-10-26 00:00:00-04:00      0.029824     -0.025782      0.042976   
2017-10-27 00:00:00-04:00     -0.004665     -0.010149      0.025267   

                                     PG  
                          Daily_Returns  
time                                     
2017-10-23 00:00:00-04:00     -0.010542  
2017-10-24 00:00:00-04:00     -0.003322  
2017-10-25 00:00:00-04:00     -0.001954  
2017-10-26 00:00:00-04:00      0.007601  
2017-10-27 00:00:00-04:00     -0.005258  

[5 rows x 60 columns]

In [49]:
# new_merged_stock_df = merged_stock_df[l]
#     if x > 0:
#         print("profit")
#     else:
#         print("loss")

In [50]:
#iShares Russell 1000 Growth ETF
# iwf_data = api.get_barset('IWF',timeframe='1D', limit=1000,).df
# iwf_data.head()

In [51]:
#iShares 1-5 year Investment grade bond
# igsb_data = api.get_barset('IGSB',timeframe='1D', limit=1000,).df
# igsb_data.head()

In [52]:
#iShares US Treasury Bond ETF
# govt_data = api.get_barset('GOVT',timeframe='1D', limit=1000,).df
# govt_data.head()